<a href="https://colab.research.google.com/github/cchadj/CodingConnected.Traci/blob/master/chad_normal_est.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import random

In [0]:
def random_points_triangle(v1, v2, v3, num):
  points = np.zeros((num, 3))
  for i in range(num):
    w1 = random.random()
    w2 = random.random() * (1.0 - w1)
    w3 = 1 - (w1 + w2)
    points[i, :] = v1 * w1 + v2 * w2 + v3 * w3

  return points
  
def add_line(fig, line, color=(1, 0, 0)):
  fig.add_trace(
    go.Scatter3d(x=line[:, 0], y=line[:, 1],  z=line[:, 2], 
                 marker=dict(
                     size=4,
                     color=color,
                     colorscale='Viridis',
                ),
                line=dict(color='darkblue',width=5)
                )
    )
  
def create_corner(angle=90):
  angle_rad = np.deg2rad(angle)

  rot = Rotation.from_rotvec([0, -angle_rad, 0])

  ax1 = np.zeros((2,3))
  ax1[1, :] = [1, 0, 0]

  ax2 = rot.apply(ax1.copy())
  ax2n = ax2[1, :] / np.linalg.norm(ax2[1, :])
  ax2[1, :] = ax2n

  ax3dir = (ax1[1, :] +   ax2[1, :]) / 2
  ax3 = np.zeros((2, 3))

  if (angle <= 90):
    rot_w = angle / 90
    ax3[1, :] = ((1 - rot_w) * ax3dir + rot_w * np.cross(ax1[1, :], ax2[1, :])) / 2
  else:
    rot_w = 1 - ((angle - 90) / 90)
    ax3[1, :] = ((1 - rot_w) * -ax3dir + rot_w * np.cross(ax1[1, :], ax2[1, :])) / 2 
  
  ax3[1, :] = ax3[1, :] / np.linalg.norm(ax3[1, :])

  return ax1, ax2, ax3


In [79]:
def create_neighborhood(n_points=1500, angle=90):
  ax1, ax2, ax3 = create_corner(angle)

  p1 = np.array([0, 0, 0])
  # triangle 1
  p2 = ax1[1, :]
  p3 = ax2[1, :]
  n1 = np.cross(ax2[1, :], ax1[1, :])
  n1 /= np.linalg.norm(n1)
  points = random_points_triangle(p1, p2, p3, 500)
  normals = np.zeros_like(points)
  normals[:] = n1
  print(normals.shape)
  print(n1)
  
  # triangle 2
  p2 = ax1[1, :]
  p3 = ax3[1, :]
  n2 = -np.cross(ax3[1, :], ax1[1, :])
  n2 /= np.linalg.norm(n2)
  points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))
  normals = np.vstack((normals, np.ones([500, 3])* n2))
  print(normals.shape)
  print(n2)
  
  # triangle 2
  p2 = ax2[1, :]
  p3 = ax3[1, :]
  n3 = -np.cross(ax2[1, :], ax3[1, :])
  n3 /= np.linalg.norm(n3)
  points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))
  normals = np.vstack((normals, np.ones([500, 3]) * n3))
  print(normals.shape)
  print(n3)

  return points, normals
  
# fig = go.Figure()

# add_line(fig, ax1)
# add_line(fig, ax2)
# add_line(fig, ax3)

points, normals = create_neighborhood(angle=60)
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1],  z=points[:, 2]))

n = points.shape[0]
normal_line_1 = np.zeros([2, 3])
normal_line_1[0, :] = np.mean(points[:int(np.floor(n/3)), :], axis=0)
normal_line_1[1, :] = normal_line_1[0, :] + normals[250]
print(normal_line_1)
print(normals[250])
add_line(fig, normal_line_1)

normal_line_2 = np.zeros([2, 3])
normal_line_2[0, :] = np.mean(points[int(np.round(n/3)):int(np.floor((2*n)/3)), :], axis=0)
normal_line_2[1, :] = normal_line_2[0, :] + normals[500]
print(normals[500])
add_line(fig, normal_line_2)

normal_line_3 = np.zeros([2, 3])
normal_line_3[0, :] = np.mean(points[int(np.round((2*n)/3)):, :], axis=0)
normal_line_3[1, :] = normal_line_3[0, :] + normals[1250]
print(normals[1250])
add_line(fig, normal_line_3)

fig.show()


(500, 3)
[0. 1. 0.]
(1000, 3)
[ 0.         -0.24253563 -0.9701425 ]
(1500, 3)
[-0.84016805 -0.24253563  0.48507125]
[[0.35782238 0.         0.20138893]
 [0.35782238 1.         0.20138893]]
[0. 1. 0.]
[ 0.         -0.24253563 -0.9701425 ]
[-0.84016805 -0.24253563  0.48507125]


In [74]:
np.mean(points[:int(n/3), :], axis=0)

array([0.37626932, 0.        , 0.2196717 ])

In [36]:
normals.shape

(3000, 3)

In [33]:
normals.shape

(3000, 3)

In [30]:
normals.shape

(3000, 3)

In [23]:
ax1, ax2, ax3 = create_corner(120)

fig = go.Figure()

p1 = np.array([0, 0, 0])
# triangle 1
p2 = ax1[1, :]
p3 = ax2[1, :]
points = random_points_triangle(p1, p2, p3, 500)

# triangle 2
p2 = ax1[1, :]
p3 = ax3[1, :]
points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))

# triangle 2
p2 = ax2[1, :]
p3 = ax3[1, :]
points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))

add_line(fig, ax1)
add_line(fig, ax2)
add_line(fig, ax3)

fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1],  z=points[:, 2]))

fig.show()

In [0]:
ax1, ax2, ax3 = create_corner(40)

fig = go.Figure()

p1 = np.array([0, 0, 0])
# triangle 1
p2 = ax1[1, :]
p3 = ax2[1, :]
points = random_points_triangle(p1, p2, p3, 500)

# triangle 2
p2 = ax1[1, :]
p3 = ax3[1, :]
points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))

# triangle 2
p2 = ax2[1, :]
p3 = ax3[1, :]
points = np.vstack((points, random_points_triangle(p1, p2, p3, 500)))

add_line(fig, ax1)
add_line(fig, ax2)
add_line(fig, ax3)

fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1],  z=points[:, 2]))

fig.show()